# Top2Vec 을 활용한 토픽모델링

In [ ]:
#importing libraries
import pandas as pd
from datetime import datetime
import numpy as np
import scipy as sp
import sklearn
import sys
import os

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from scipy import linalg

# !pip install gensim 완료
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import pickle
import string
import time


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
path = r'/gdrive/MyDrive/석사 수업/2학기/비즈니스 텍스트마이닝/diabetes_SA'
os.chdir(path)

with open('neg_doc','rb') as fr:
  df = pickle.load(fr)
df

,title,content,writer,date,view,likes,no_comments,board,url,year,month,hour,text_content,text_title,content_sep,title_sep,content_p,title_p,content_joined,neg,pos,sent_pred
0,\n 약 부작용이 있으면 어떻게 하나요??\n,\n관리를 못해 당화혈색소가 8.9가 됐더라구요ㅜ원래먹던약이 직듀오서방정 10/10...,\n 긍정왕썰\n,2021-05-14 18:42:00,157,0.0,8.0,all_couns,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2021,5,18,관리를 못해 당화혈색소가 됐더라구요ㅜ원래먹던약이 직듀오서방정 디아미크롱서방정...,약 부작용이 있으면 어떻게 하나요,[관리를 못해 당화혈 색소가 됐더라구 요ㅜ원래 먹던 약이 직 듀오 서방 정 디아미크...,[약 부작용이 있으면 어떻게 하나요],"[관리, 당화, 혈, 색소, 더, 원래, 먹다, 약, 직, 듀오, 서방, 정, 디,...","[약, 부작용, 어떻다, 하나요]",관리 당화 혈 색소 더 원래 먹다 약 직 듀오 서방 정 디 아미 크롱 방정 이번 새...,1.570287,-1.855057,0
1,\n 안녕하세요~ 당뇨기계 추천 받고자 합니다\n,\n어머니가 오래동안 당뇨를 가지고 계신데 기계가 오래되어 바꿔 드리고 싶은데 체혈...,\n 티타2\n,2021-05-14 18:30:00,101,0.0,8.0,all_couns,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2021,5,18,어머니가 오래동안 당뇨를 가지고 계신데 기계가 오래되어 바꿔 드리고 싶은데 체혈 ...,안녕하세요 당뇨기계 추천 받고자 합니다,[어머니가 오래 동안 당뇨를 가지고 계신데 기계가 오래되어 바꿔 드리고 싶은데 체혈...,"[안녕하세요, 당뇨기계 추천 받고 자 합니다]","[어머니, 당뇨, 가지, 계시다, 기계, 오래되다, 바꾸다, 드리다, 싶다, 채혈,...","[안녕하다, 당뇨, 기계, 추천, 받다, 자]",어머니 당뇨 가지 계시다 기계 오래되다 바꾸다 드리다 싶다 채혈 아프다 않다 시험지...,0.183513,-0.491494,0
2,\n 체중과의 사투를 벌이고 있네요ㅠ\n,\n확진후8개월만에 10키로가 내의지와는 전혀상관없이 체중이 빠져버렸네요 ​마지노선...,\n 천왕봉\n,2021-05-14 17:27:00,221,0.0,16.0,all_couns,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2021,5,17,내의지와는 전혀상관없이 체중이 빠져버렸네요 ​마지노선인 사수하기 위해 ...,체중과의 사투를 벌이고 있네요ㅠ,"[내의지와는 전혀 상관없이 체중이 빠져버렸네요, 마지노선인 사수하기 위해 먹던 잡곡...",[체중과의 사투를 벌이고 있네요ㅠ],"[의지, 전혀, 체중, 빠지다, 버리다, 마지노선, 사수, 위해, 먹다, 잡곡, 밥...","[체중, 사투, 벌이]",의지 전혀 체중 빠지다 버리다 마지노선 사수 위해 먹다 잡곡 밥 오늘 그램 올리다 ...,2.397109,-2.645771,0
3,\n 쌀국수\n,\n안녕들하세요^^점심에 쌀국수를 먹었네요 ㅋㅋ높을거라 생각은 했지만 ㅠㅠ1시간 후...,\n 희뿡2\n,2021-05-14 16:02:00,107,0.0,4.0,all_couns,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2021,5,16,안녕들하세요^^점심에 쌀국수를 먹었네요 ㅋㅋ높을거라 생각은 했지만 후 후 ^^...,쌀국수,"[안녕들 하세요 점심에 쌀국수를 먹었네요 ㅋㅋ, 높을 거라 생각은 했지만 후 후 쌀...",[쌀국수],"[점심, 쌀국수, 먹다, 높다, 생각, 후, 후, 쌀국수, 거리, 두기, 공복, 어렵다]",[쌀국수],점심 쌀국수 먹다 높다 생각 후 후 쌀국수 거리 두기 공복 어렵다,-0.143517,-0.179952,0
4,\n 운동하면 오히려 왜혈당이 오르나요?\n,\n예전엔안그랬는데당뇨약먹고 부터는 운동 하면 혈당이더오르네요?오늘같은 경우 식전에...,\n 제라툴\n,2021-05-14 15:58:00,293,0.0,4.0,all_couns,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2021,5,15,예전엔안그랬는데당뇨약먹고 부터는 운동 하면 혈당이더오르네요 오늘같은 경우 식전에 ...,운동하면 오히려 왜혈당이 오르나요,"[예전엔 안 그랬는데 당뇨약 먹고 부터는 운동 하면 혈당이 더 오르네요, 오늘 같은...",[운동하면 오히려 왜 혈당이 오르나요],"[예전, 안, 당뇨, 약, 운동, 혈당, 더, 오르다, 오늘, 경우, 식, 전, 다...","[운동, 오히려, 왜, 혈당, 오르다]",예전 안 당뇨 약 운동 혈당 더 오르다 오늘 경우 식 전 다음 혈당 재 으레 나오다...,0.201569,-0.536045,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23137,\n 26일 힘내라\n,≫아침몸무게/저녁몸무게 :68kg,\n 힘내라 나야\n,2010-08-26 15:05:00,53,0.0,2.0,exerc,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2010,8,15,≫아침몸무게 저녁몸무게,힘내라,[≫아침 몸무게 저녁 몸무게],[힘내라],"[아침, 몸무게, 저녁, 몸무게]",[힘내다],아침 몸무게 저녁 몸무게,-0.151557,-0.207595,0
23138,\n [웃짜 8월25일 -7일차 71.6kg] 이소라 다이어트 ...,≫목표몸무게/현재몸무게 : 54kg / 71.6(표준체중 + 17.6kg),\n 웃짜\n,2010-08-25 22:51:00,171,0.0,3.0,exerc,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2010,8,22,≫목표몸무게 현재몸무게 표준체중,이소라 다이어트 체조 그리고 혈당기,[≫ 목표 몸무게 현재 몸무게 표준체중],[이소라 다이어트 체조 그리고 혈당기],"[목표, 몸무게, 현재, 몸무게, 표준, 체중]","[이소라, 다이어트, 체조, 혈당]",목표 몸무게 현재 몸무게 표준 체중,1.245663,-1.536207,0
23139,\n 8/25 나나의 입원일기 3일째(500->140)\n ...,"입원 1,2일째에는...",\n 나나\n,2010-08-25 19:23:00,210,0.0,5.0,exerc,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2010,8,19,입원,나나의 입원일기,[입원],[나나의 입원일기],[입원],"[나나, 입원, 일기]",입원,0.495256,-0.806772,0
23140,\n 8/23 나나의 입원일기 1일째\n,새벽 6시 반에 일어나,\n 나나\n,2010-08-25 09:49:00,214,0.0,6.0,exerc,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2010,8,9,새벽 반에 일어나,나나의 입원일기,[새벽 반에 일어나],[나나의 입원일기],"[새벽, 반, 일다]","[나나, 입원, 일기]",새벽 반 일다,1.650019,-1.948385,0


In [ ]:
# !pip install top2vec

from top2vec import Top2Vec

In [ ]:
# content
# word to a str
content_joined = df['content_p'].apply(lambda x: " ".join(x)) # input은  [docu1,docu2] 의 형태. 
content_joined_list = list(content_joined)
content_joined_list[0]
#wc_input = joined.sum()

'관리 당화 혈 색소 더 원래 먹다 약 직 듀오 서방 정 디 아미 크롱 방정 이번 새다 추가 게 리 피토 엠 서방 정 약 바꾸다 첫날 두통 심하다 오가다 그 다음 날 약 설사 속 쓰리다 소화 불량 생기 식욕 없어지다 그렇다 다른 건 속 쓰리다 참다 힘들다 달 뒤 관리 다시 가기 전 가야'

In [ ]:
#title
# word to a str
joined_title = df['title_p'].apply(lambda x: " ".join(x)) # input은  [docu1,docu2] 의 형태. 
title_joined_list = list(joined_title)
title_joined_list[0]
#wc_input = joined.sum()

'약 부작용 어떻다 하나요'

In [ ]:
model = Top2Vec(documents=title_joined_list, speed="learn", workers=8, min_count = 75) #min_counts = 바꿔볼 수도.

2021-05-30 12:48:28,165 - top2vec - INFO - Pre-processing documents for training
2021-05-30 12:48:29,038 - top2vec - INFO - Creating joint document/word embedding
2021-05-30 12:50:20,131 - top2vec - INFO - Creating lower dimension embedding of documents
2021-05-30 12:50:38,651 - top2vec - INFO - Finding dense areas of documents
2021-05-30 12:50:40,685 - top2vec - INFO - Finding topics


In [ ]:
# !pip install tensorflow tensorflow_hub tensorflow_text

In [ ]:
model.get_num_topics() # 4개의 토픽 생성.

3

In [ ]:
model.get_topic_sizes()

(array([8669, 7407, 7066]), array([0, 1, 2]))

In [ ]:
topic_words, word_scores, topic_nums = model.get_topics(3)


In [ ]:
# 제목을 topic modeling.
topic_words

array([['먹다', '검사', '병원', '없다', '부탁드리다', '혈당', '경우', '식사', '결과', '들다',
        '조절', '수치', '고민', '주다', '가다', '어떻다', '먹이다', '계시다', '높다', '저녁',
        '공복', '체크', '타다', '다시', '내과', '시간', '자전거', '봐주다', '정상', '오늘',
        '추천', '바꾸다', '드리다', '차이', '빠지다', '드시다', '당화', '지다', '식단', '이렇다',
        '때문', '좋다', '혈압', '식후', '계속', '맞다', '임신', '내일', '복용', '혹시'],
       ['당화', '떨어지다', '마감', '대해', '처음', '당뇨', '아침', '정말', '알려주다', '보신',
        '조언', '아큐첵', '해주다', '결과', '추천', '재다', '올리다', '보험', '현미', '확진',
        '지다', '체크', '걸다', '사다', '계속', '걷기', '도움', '착하다', '방법', '어제',
        '진단', '하루', '내과', '관리', '싶다', '금일', '드리다', '경우', '점심', '당화혈색소',
        '먹이다', '식전', '측정', '먹다', '식후', '봐주다', '효과', '쓰다', '하나', '받다'],
       ['먹다', '대해', '받다', '하나', '갑자기', '현미', '혈압', '먹이다', '오늘', '샐러드',
        '봐주다', '확진', '좋다', '계속', '들다', '정상', '측정기', '경우', '꾸다', '혈당',
        '내일', '지다', '당화혈색소', '높다', '고민', '당뇨', '내과', '타다', '요즘', '빠지다',
        '되다', '어떻다', '추천', '조절', '계시다', '결과', '질문', '도와주다', '정말', '보험',
      

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=0, num_docs=10)


In [ ]:
# topic에 해당하는 확인
documents

array(['저녁 약속 이제 끝 건가', '금일 당화혈색소 마감 혈당', '요즘 예식장 뷔페 음식',
       '당화 혈 검사 겸 어지러움 때문 병원 검사 가다 오다', '당뇨 약 바꾸다 낫다 혈당 조절 안',
       '혈압 대해 자다 아시 분', '정말 초보 질문 먹다 싶다', '합병증 언제 오다', '채단 타다 단 채 타다',
       '젠 운동 시간대 바꾸다'], dtype=object)

In [ ]:
topic_nums

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
topic_words

[array(['혈당', '식후', '공복', '피크', '들어오다', '시간', '보통', '밀리다', '나오다', '이해',
        '높다', '정도', '거의', '나와다', '범위', '걸다', '정상', '정상인', '해보다', '건가',
        '유지', '기준', '식이', '기저', '그렇다', '의미', '재는', '숫자', '올라가다', '수치',
        '이렇다', '평균', '나름', '지나서다', '관리', '넘다', '전날', '측정', '튀다', '저녁',
        '인슐린', '차이', '경우', '넘기다', '오르다', '체크', '늘리다', '취침', '어떻다', '아예'],
       dtype='<U5'),
 array(['평균', '공복', '혈당', '측정기', '안녕하다', '당화혈색소', '당화', '지난', '측정', '오차',
        '나오다', '범위', '차이', '기기', '내려가다', '유지', '아큐첵', '높다', '지방간', '재다',
        '기존', '비교', '자가', '진단', '정상', '처방', '식후', '의사', '퍼포머', '검사', '작년',
        '관리', '기준', '식이', '최근', '혈색소', '감량', '해보다', '이번', '건강검진', '재는',
        '채혈', '바꾸다', '중인', '확진', '기계', '처음', '다시', '수치', '숫자'], dtype='<U5'),
 array(['투여', '인슐린', '맞다', '입원', '경우', '혈당', '상황', '지속', '설명', '지금', '상태',
        '이해', '의사', '식후', '처방', '어렵다', '건가', '시작', '주사', '잡히다', '이하',
        '이렇다', '어떻다', '문의', '지다', '조절', '높다', '약물', '이후', '길다', '중이', '병원',
        '다른', '퇴원', '언제', '

# TOP2VEC 이 안된다면.. LDA

In [ ]:
## sklearn에서 LDA
 
    
# Document - Term Matrix만들기
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df=2)

# apply transformation
df['title_n_str']=[' '.join(token) for token in df['content_p']]
tf = vectorizer.fit_transform(df['title_n_str']).toarray()
# sklearn LDA는 numpy array형태로 들어가기에!



# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()


from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 10

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model.fit(tf)

def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 5
topic_dictionary=display_topics(model, tf_feature_names, no_top_words)



In [ ]:
topic_dictionary

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,혈당,8642.9,먹다,8277.5,당뇨,3216.2,건강,695.0,혈당,1564.4,먹다,2872.1,먹다,3797.0,쓰다,580.1,운동,3035.1,아프다,342.6
1,공복,4949.4,넣다,1149.1,검사,2642.2,좋다,529.5,맞다,1097.0,당뇨,1666.1,오늘,875.1,오늘,303.7,오늘,677.3,없다,335.7
2,먹다,4380.5,혈당,1127.9,병원,2620.8,효과,377.8,인슐린,1048.5,관리,1287.5,들다,835.1,대다,226.3,걷다,584.8,맞다,331.4
3,나오다,4286.6,아침,988.2,받다,2392.6,비타민,313.9,당뇨,719.7,운동,1189.1,오다,776.5,가다,221.0,걷기,565.6,가다,320.5
4,수치,3031.2,점심,917.3,가다,1449.3,히다,301.7,받다,667.9,않다,989.2,없다,668.0,사람,205.2,자전거,529.8,들다,257.9
